In [ ]:
!nvidia-smi

In [1]:
!sudo apt update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,375 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,125 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-backport

In [2]:
!sudo apt purge *nvidia* -y

Streaming output truncated to the last 5000 lines.
Package 'linux-objects-nvidia-450-server-5.15.0-1049-gcp' is not installed, so not removed
Package 'linux-objects-nvidia-450-server-5.15.0-1049-intel-iotg' is not installed, so not removed
Package 'linux-objects-nvidia-450-server-5.15.0-1049-oracle' is not installed, so not removed
Package 'linux-objects-nvidia-450-server-5.15.0-105-generic' is not installed, so not removed
Package 'linux-objects-nvidia-450-server-5.15.0-105-lowlatency' is not installed, so not removed
Package 'linux-objects-nvidia-450-server-5.15.0-1050-aws' is not installed, so not removed
Package 'linux-objects-nvidia-450-server-5.15.0-1050-azure' is not installed, so not removed
Package 'linux-objects-nvidia-450-server-5.15.0-1050-intel-iotg' is not installed, so not removed
Package 'linux-objects-nvidia-450-server-5.15.0-1050-oracle' is not installed, so not removed
Package 'linux-objects-nvidia-450-server-5.15.0-1051-aws' is not installed, so not removed
Package 

In [3]:
!sudo apt install nvidia-driver-530 -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages were automatically installed and are no longer required:
  ocl-icd-opencl-dev opencl-c-headers opencl-clhpp-headers
Use 'sudo apt autoremove' to remove them.
The following additional packages will be installed:
  cpp-12 dctrl-tools dkms fakeroot gcc-12 keyboard-configuration libasan8
  libfakeroot libfontenc1 libgcc-12-dev libjansson4 liblocale-gettext-perl
  libnvidia-cfg1-555 libnvidia-common-555 libnvidia-compute-555
  libnvidia-decode-555 libnvidia-encode-555 libnvidia-extra-555
  libnvidia-fbc1-555 libnvidia-gl-555 libtsan2 libudev1 libxcvt0 libxfont2
  libxkbfile1 nvidia-compute-utils-555 nvidia-dkms-555 nvidia-driver-555
  nvidia-firmware-555-555.42.02 nvidia-kernel-common-555
  nvidia-kernel-source-555 nvidia-prime nvidia-settings nvidia-utils-555
  python3-xkit screen-resolution-extra systemd-hwe-hwdb udev x11-xkb-utils
  xcvt xfonts-base xfonts-encodings xfo

In [4]:
!cinfo

/bin/bash: line 1: cinfo: command not found


In [5]:
!apt-get install opencl-headers

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following package was automatically installed and is no longer required:
  ocl-icd-opencl-dev
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  opencl-headers
0 upgraded, 1 newly installed, 0 to remove and 44 not upgraded.
Need to get 1,754 B of archives.
After this operation, 12.3 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 opencl-headers all 3.0~2022.01.04-1 [1,754 B]
Fetched 1,754 B in 1s (2,200 B/s)
Selecting previously unselected package opencl-headers.
(Reading database ... 123831 files and directories currently installed.)
Preparing to unpack .../opencl-headers_3.0~2022.01.04-1_all.deb ...
Unpacking opencl-headers (3.0~2022.01.04-1) ...
Setting up opencl-headers (3.0~2022.01.04-1) ...


In [6]:
!apt-get install ocl-icd-liopencl1

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package ocl-icd-liopencl1


In [7]:
!sudo find /usr/ -name cl.h 2>/dev/null

/usr/include/CL/cl.h
/usr/local/lib/python3.10/dist-packages/tensorflow/include/third_party/gpus/cuda/include/CL/cl.h
/usr/local/cuda-12.2/targets/x86_64-linux/include/CL/cl.h


In [8]:
%%writefile opencl_hello_world.c
#include <stdio.h>
#include <stdlib.h>
#include <CL/cl.h>


#define CL_USE_DEPRECATED_OPENCL_1_1_APIS


// OpenCL kernel code
const char *kernelSource =
"__kernel void helloWorld() {\n"
"    printf(\"Hello, World!\\n\");\n"
"}\n";

int main() {
    cl_platform_id platform;
    cl_device_id device;
    cl_context context;
    cl_command_queue queue;
    cl_program program;
    cl_kernel kernel;


    // Step 1 : Discover Platforms and Devices
    clGetPlatformIDs(1, &platform, NULL);
    clGetDeviceIDs(platform, CL_DEVICE_TYPE_GPU, 1, &device, NULL);

    // Step 2 : Create Context and Command Queue
    context = clCreateContext(NULL, 1, &device, NULL, NULL, NULL);
    queue = clCreateCommandQueue(context, device, 0, NULL);

    // Step 3 : Create and Build Program
    program = clCreateProgramWithSource(context, 1, (const char**)&kernelSource, NULL, NULL);
    clBuildProgram(program, 1, &device, NULL, NULL, NULL);

    // Step 4 : Create kernel
    kernel = clCreateKernel(program, "helloWorld", NULL);

    // Step 5 : Enqueue Kernel for Execution
    clEnqueueTask(queue, kernel, 0, NULL, NULL);

    // Step 6 : Release Resources
    clReleaseKernel(kernel);
    clReleaseProgram(program);
    clReleaseCommandQueue(queue);
    clReleaseContext(context);


    return 0;

}

Writing opencl_hello_world.c


In [9]:
!gcc -o hello_world opencl_hello_world.c -I "usr/include/CL" -L "/usr/include/CL" -lOpenCL

In file included from /usr/include/CL/cl.h:20,
                 from opencl_hello_world.c:3:
/usr/include/CL/cl_version.h:22:9: note: ‘#pragma message: cl_version.h: CL_TARGET_OPENCL_VERSION is not defined. Defaulting to 300 (OpenCL 3.0)’
   22 | #pragma message("cl_version.h: CL_TARGET_OPENCL_VERSION is not defined. Defaulting to 300 (OpenCL 3.0)")
      |         ^~~~~~~
opencl_hello_world.c: In function ‘main’:
opencl_hello_world.c:30:5: warning: ‘clCreateCommandQueue’ is deprecated []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wdeprecated-declarations-Wdeprecated-declarations]8;;]
   30 |     queue = clCreateCommandQueue(context, device, 0, NULL);
      |     ^~~~~
In file included from opencl_hello_world.c:3:
/usr/include/CL/cl.h:1906:1: note: declared here
 1906 | clCreateCommandQueue(cl_context                     context,
      | ^~~~~~~~~~~~~~~~~~~~
opencl_hello_world.c:40:5: warning: ‘clEnqueueTask’ is deprecated []8;;https://gcc.gnu.org/onlinedocs/g

In [10]:
!./hello_world

Hello, World!
